# Módulos e versões

In [ ]:
python_version = !python -V     #version 3.7.10
import pandas as pd             #version 1.1.5
import numpy as np              #version 1.19.5
import seaborn as sns           #version 0.11.1
import matplotlib as plt        #version 3.2.2

#normalizando os dados
from sklearn.preprocessing import MinMaxScaler

In [ ]:
print(f'Python version - {python_version[0][7:]}')
print(f'Pandas version - {pd.__version__}')
print(f'Numpy version - {np.__version__}')
print(f'Seaborn version - {sns.__version__}')
print(f'Matplotlib version - {plt.__version__}')

Python version - 3.7.11
Pandas version - 1.1.5
Numpy version - 1.19.5
Seaborn version - 0.11.1
Matplotlib version - 3.2.2


# Funções

In [ ]:
def transforma_df(dados_em_bytes):
    #transforma conteúdo da url em string e faz um split
    dados_str = str(dados_em_bytes.content, 'utf-8')
    dados_str = dados_str.split('\n')

    #itera para criar as listas
    contador = 0
    lista_date = []
    lista_open = []
    lista_high = []
    lista_low = []
    lista_close = []
    lista_volume = []
    for item in dados_str:
        separa_item = str(dados_str[contador]).split(',')
        lista_date.append(separa_item[0])
        lista_open.append(separa_item[1])
        lista_high.append(separa_item[2])
        lista_low.append(separa_item[3])
        lista_close.append(separa_item[4])
        lista_volume.append(separa_item[6])
        contador += 1

    #dicionário com chave na primeira linha e resultados nas restantes
    dicionario_resultados = {
        lista_date[0]:lista_date[1:],
        lista_open[0]:lista_open[1:],
        lista_high[0]:lista_high[1:],
        lista_low[0]:lista_low[1:],
        lista_close[0]:lista_close[1:],
        lista_volume[0]:lista_volume[1:]
    }
    
    #retorna um DataFrame em pandas
    return pd.DataFrame(data=dicionario_resultados)

# Extração das bases

## Arquivos

In [ ]:
tesla = pd.read_csv('/content/drive/MyDrive/projeto_acoes/tesla.csv')
apple = pd.read_csv('/content/drive/MyDrive/projeto_acoes/apple.csv')
amazon = pd.read_csv('/content/drive/MyDrive/projeto_acoes/amazon.csv')
google = pd.read_csv('/content/drive/MyDrive/projeto_acoes/google.csv')
microsoft = pd.read_csv('/content/drive/MyDrive/projeto_acoes/microsoft.csv')
facebook = pd.read_csv('/content/drive/MyDrive/projeto_acoes/facebook.csv')
tsmc = pd.read_csv('/content/drive/MyDrive/projeto_acoes/tsmc.csv')
tencent = pd.read_csv('/content/drive/MyDrive/projeto_acoes/tencent.csv')
alibaba = pd.read_csv('/content/drive/MyDrive/projeto_acoes/alibaba.csv')
nvidia = pd.read_csv('/content/drive/MyDrive/projeto_acoes/nvidia.csv')
samsung = pd.read_csv('/content/drive/MyDrive/projeto_acoes/samsung.csv')
paypal = pd.read_csv('/content/drive/MyDrive/projeto_acoes/paypal.csv')
asml = pd.read_csv('/content/drive/MyDrive/projeto_acoes/asml.csv')
adobe = pd.read_csv('/content/drive/MyDrive/projeto_acoes/adobe.csv')
oracle = pd.read_csv('/content/drive/MyDrive/projeto_acoes/oracle.csv')

## Cria dicionário para acessar os objetos DataFrame

In [ ]:
empresas = {
    'tesla':tesla,
    'apple':apple,
    'amazon':amazon,
    'google':google,
    'microsoft':microsoft,
    'facebook':facebook,
    'tsmc':tsmc,
    'tencent':tencent,
    'alibaba':alibaba,
    'nvidia':nvidia,
    'samsung':samsung,
    'paypal':paypal,
    'asml':asml,
    'adobe':adobe,
    'oracle':oracle
}

# Transformação das bases

### Renomeia colunas

In [ ]:
dict_columns = {}
for empresa, df in empresas.items():
    for item in df.columns:
        dict_temp = {item: item.lower()}
        dict_columns.update(dict_temp)
    df.rename(columns=dict_columns, inplace=True)

### Transforma coluna date em datetime

In [ ]:
for empresa, df in empresas.items():
    df['date'] = pd.to_datetime(df['date'])

### Seleciona o mesmo range de datas para o dataframe

In [ ]:
lista_data = []
for empresa, df in empresas.items():
    data_inicial = df['date'].iloc[0]
    lista_data.append(data_inicial)

data_minima = max(lista_data)
for empresa, df in empresas.items():
    indice = df.query('date<@data_minima').index
    df.drop(index=indice, inplace=True)
    df.reset_index(inplace=True, drop=True)

# Funções

## Criar dataframe com os dados pretendidos

In [ ]:
def df_previsao(df_target, col_target, col_feature):
    #cria estruturas de dados
    dict_df = {}
    lista_empresas = list(empresas.keys())
    #itera para separar os dados da forma selecionada
    for empresa in lista_empresas:
    #cria coluna target e salva em um dicionário
        if empresa == df_target:
            target = empresas[df_target]
            target_column_name = str('target_' + df_target + '_' + col_target)
            dict_target = {
                'date':target['date'],
                target_column_name:target[col_target]
                }
    #cria as colunas de feature e salva em um dicionário
        else:
            feature = empresas[empresa][col_feature]
            feature_column_name = str('feature_' + empresa + '_' + col_feature)
            dict_feature = {
                feature_column_name:feature
            }
            dict_df.update(dict_feature)
    #cria dicionário com todas as colunas e o objeto DataFrame
    dict_df.update(dict_target)
    df = pd.DataFrame(dict_df)
    #reestrutura a ordem das colunas
    column_date = df['date']
    df.drop(columns=['date'], inplace=True)
    df.insert(loc=0, column='date', value=column_date)
    return df

## Calcula correlação e exclui features pouco correlacionadas

In [ ]:
def drop_correlacao(dataframe, col_target, drop_correlacao='media', grafico=False):
    df_corr = dataframe.corr()
    contador = 0
    corr_fraca = []
    corr_media = []
    corr_forte = []
    for i in range(df_corr.shape[0]):
        indice = df_corr[col_target].index
        valor = df_corr[col_target].values
        if valor[contador] <= -0.75 or valor[contador] >= 0.75:
            corr_forte.append(indice[contador])
        elif valor[contador] <= -0.5 or valor[contador] >= 0.5:
            corr_media.append(indice[contador])
        else:
            corr_fraca.append(indice[contador])
        contador += 1

    if drop_correlacao=='media':
        print(f'Forte correlação: {corr_forte}\nMédia correlação [excluída]: {corr_media}\nFraca correlação [excluída]: {corr_fraca}')
        dataframe.drop(columns=corr_fraca, inplace=True)
        dataframe.drop(columns=corr_media, inplace=True)
        return dataframe
    elif drop_correlacao=='fraca':
        print(f'Forte correlação: {corr_forte}\nMédia correlação: {corr_media}\nFraca correlação [excluída]: {corr_fraca}')
        dataframe.drop(columns=corr_fraca, inplace=True)
        return dataframe
    else:
        print('ERRO')
    

## Feature engineering

In [ ]:
def modelagem(dataframe, diff=True, estatistica='media', periodos=5, drop=True):
    colunas_originais = dataframe.columns[1:len(dataframe.columns) - 1]
    if diff is True:
        for coluna in colunas_originais:
            novo_nome = coluna + '_diff'
            posicao = dataframe.columns.to_list()
            dataframe.insert(column=novo_nome, loc=posicao.index(coluna) + 1, value=dataframe[coluna].diff())
    else:
        pass
    if estatistica=='media':
        for coluna in colunas_originais:
            novo_nome = coluna + '_media_' + str(periodos)
            posicao = dataframe.columns.to_list()
            dataframe.insert(column=novo_nome, loc=posicao.index(coluna) + 2, value=dataframe[coluna].rolling(periodos).mean())
    elif estatistica=='soma':
        for coluna in colunas_originais:
            novo_nome = coluna + '_soma_' + str(periodos)
            posicao = dataframe.columns.to_list()
            dataframe.insert(column=novo_nome, loc=posicao.index(coluna) + 2, value=dataframe[coluna].rolling(periodos).sum())
    elif estatistica=='mediana':
        for coluna in colunas_originais:
            novo_nome = coluna + '_mediana_' + str(periodos)
            posicao = dataframe.columns.to_list()
            dataframe.insert(column=novo_nome, loc=posicao.index(coluna) + 2, value=dataframe[coluna].rolling(periodos).median())
    else:
        pass
    if drop is True:
        return dataframe.dropna()
    else:
        return dataframe   

# Análises

## Transformações DataFrame

In [ ]:
df_model = df_previsao(df_target='facebook', col_target='close', col_feature='open')
df_model = drop_correlacao(dataframe=df_model, col_target='target_facebook_close', drop_correlacao='media')
df_model = modelagem(df_model, diff=True, estatistica='mediana', periodos=20)
df_model

Forte correlação: ['feature_google_open', 'feature_microsoft_open', 'feature_nvidia_open', 'feature_asml_open', 'feature_adobe_open', 'feature_oracle_open', 'target_facebook_close']
Média correlação [excluída]: ['feature_apple_open', 'feature_amazon_open', 'feature_alibaba_open', 'feature_paypal_open']
Fraca correlação [excluída]: ['feature_tesla_open', 'feature_tsmc_open', 'feature_tencent_open', 'feature_samsung_open']


,date,feature_google_open,feature_google_open_diff,feature_google_open_mediana_20,feature_microsoft_open,feature_microsoft_open_diff,feature_microsoft_open_mediana_20,feature_nvidia_open,feature_nvidia_open_diff,feature_nvidia_open_mediana_20,feature_asml_open,feature_asml_open_diff,feature_asml_open_mediana_20,feature_adobe_open,feature_adobe_open_diff,feature_adobe_open_mediana_20,feature_oracle_open,feature_oracle_open_diff,feature_oracle_open_mediana_20,target_facebook_close
19,2020-09-10,1560.640015,3.109986,1579.549988,213.399994,5.799988,213.629998,129.892502,5.892502,126.528751,364.269989,6.299988,376.619996,491.149994,17.239990,477.615006,57.439999,1.279999,56.155001,268.089996
20,2020-09-11,1536.000000,-24.640015,1579.549988,207.199997,-6.199997,213.629998,124.855003,-5.037499,126.528751,364.000000,-0.269989,376.390000,487.079987,-4.070007,482.610001,60.709999,3.270000,56.250000,266.609985
21,2020-09-14,1539.005005,3.005005,1579.549988,204.240005,-2.959992,213.629998,130.809998,5.954995,127.282501,369.769989,5.769989,376.390000,484.989990,-2.089997,486.034989,60.860001,0.150002,56.610000,266.149994
22,2020-09-15,1536.000000,-3.005005,1579.549988,208.419998,4.179993,213.629998,132.800003,1.990005,127.826252,374.549988,4.779999,376.020004,493.739990,8.750000,489.114991,60.009998,-0.850003,57.005001,272.420013
23,2020-09-16,1555.540039,19.540039,1579.549988,210.619995,2.199997,213.629998,129.572495,-3.227508,127.915001,372.000000,-2.549988,375.235000,505.500000,11.760010,492.444992,61.119999,1.110001,57.230002,263.519989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,2021-08-03,2720.000000,10.310059,2644.015014,285.420013,-0.939972,282.210006,197.399994,0.399994,196.489998,770.530029,-0.279969,716.274993,620.229980,-5.640015,608.555023,87.879997,0.149994,87.485001,351.239990
245,2021-08-04,2724.989990,4.989990,2651.500000,286.220001,0.799988,283.095001,199.899994,2.500000,196.489998,792.010010,21.479981,726.489990,622.469971,2.239991,611.645019,89.790001,1.910004,87.605000,358.920013
246,2021-08-05,2720.570068,-4.419922,2679.099976,286.880005,0.660004,284.505004,205.000000,5.100006,196.489998,805.510010,13.500000,740.394989,626.989990,4.520019,615.809998,90.180000,0.389999,87.705002,362.970001
247,2021-08-06,2725.899902,5.329834,2707.444946,288.510010,1.630005,285.295013,205.240005,0.240005,196.489998,784.500000,-21.010010,748.894989,629.270020,2.280030,618.035004,89.239998,-0.940002,87.745002,363.510010


In [ ]:
#transforma a coluna de data em índice
df_model.set_index(keys=df_model['date'], inplace=True)
df_model.drop(columns=['date'], inplace=True)
df_model

,feature_facebook_high,feature_tsmc_high,feature_nvidia_high,feature_samsung_high,feature_paypal_high,feature_adobe_high,feature_oracle_high,target_tesla_low
date,,,,,,,,
2020-08-13,265.160004,79.099998,117.217499,59600.0,195.130005,455.899994,54.200001,313.451996
2020-08-14,262.649994,79.230003,117.047501,58400.0,195.468994,452.809998,54.560001,325.328003
2020-08-17,264.100006,80.889999,124.097504,59900.0,197.419998,456.100006,54.389999,334.566010
2020-08-18,265.149994,80.820000,124.959999,59200.0,198.750000,464.500000,56.930000,369.022003
2020-08-19,267.910004,79.230003,123.150002,57600.0,196.839996,468.109985,56.799999,368.242004
...,...,...,...,...,...,...,...,...
2021-08-03,353.769989,118.639999,202.220001,82300.0,274.540009,624.700012,90.169998,701.010010
2021-08-04,360.480011,119.550003,203.179993,NaN,276.092987,627.080017,90.089996,708.929993
2021-08-05,363.899994,119.239998,207.330002,NaN,280.959991,632.469971,90.519997,711.409973


In [ ]:
#REVISAR
X = df_model.iloc[:, 0:4]
y = df_model.iloc[:, -1]

scaler = MinMaxScaler()
X_normalizado = scaler.fit_transform(X)

y_normalizado = np.asarray(y)
y_normalizado = y_normalizado.reshape(-1,1)
y_normalizado = scaler.fit_transform(y_normalizado)

In [ ]:
df_model_normalizado = pd.DataFrame(X_normalizado)
df_model_normalizado['5'] = y_normalizado
df_model_normalizado = pd.DataFrame(data=df_model_normalizado, columns=df_model.columns)

In [ ]:
X_normalizado

array([[6.44033900e-04, 6.89443235e-03, 3.00197380e-03, 5.38937519e-03],
       [2.17305145e-03, 4.93554421e-03, 7.70109879e-04, 3.57075627e-03],
       [1.99515397e-03, 4.29844646e-03, 8.63104209e-04, 1.49692553e-03],
       ...,
       [7.61741252e-01, 9.45003062e-01, 9.69213082e-01, 9.87945658e-01],
       [7.66413834e-01, 9.48185983e-01, 9.67338679e-01, 9.81024533e-01],
       [7.60851719e-01, 9.57808497e-01, 9.72287658e-01, 9.98157331e-01]])